In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!git clone https://github.com/susannapaoli/NLP-final-project.git

Cloning into 'NLP-final-project'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 608 bytes | 608.00 KiB/s, done.


In [4]:
%cd /content/NLP-final-project

/content/NLP-final-project


## Import packages 

In [4]:
!python -m spacy download en_core_web_sm
!python -m spacy download fr_core_news_sm

2023-04-14 19:21:10.515599: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-14 19:21:11.486903: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-14 19:21:12.865409: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-14 19:21:12.865857: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [5]:
import math
import time
import io
import numpy as np
import csv
from IPython.display import Image

import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.datasets import Multi30k
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import vocab
from torchtext.utils import download_from_url, extract_archive
from torch.nn.utils.rnn import pad_sequence

from tqdm import tqdm_notebook, tqdm


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You are using device: %s" % device)

You are using device: cuda


## Data preprocessing

In [7]:
MAX_LEN = 20
url_base = 'https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/'
train_urls = ('train.fr.gz', 'train.en.gz')
val_urls = ('val.fr.gz', 'val.en.gz')
test_urls = ('test_2016_flickr.fr.gz', 'test_2016_flickr.en.gz')

train_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in train_urls]
val_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in val_urls]
test_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in test_urls]

fr_tokenizer = get_tokenizer('spacy', language='fr_core_news_sm')
en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')

In [8]:
def build_vocab(filepath, tokenizer):
  counter = Counter()
  with io.open(filepath, encoding="utf8") as f:
    for string_ in f:
      counter.update(tokenizer(string_))
  return vocab(counter, specials=['<unk>', '<pad>', '<sos>', '<eos>'], min_freq=2)


fr_vocab = build_vocab(train_filepaths[0], fr_tokenizer)
en_vocab = build_vocab(train_filepaths[1], en_tokenizer)
fr_vocab.set_default_index(fr_vocab['<unk>'])
en_vocab.set_default_index(en_vocab['<unk>'])

In [9]:
def data_process(filepaths):
  raw_fr_iter = iter(io.open(filepaths[0], encoding="utf8"))
  raw_en_iter = iter(io.open(filepaths[1], encoding="utf8"))
  data = []
  for (raw_fr, raw_en) in zip(raw_fr_iter, raw_en_iter):
    raw_en_l=raw_en.lower()     #turn sentences to lower case 
    raw_fr_l=raw_fr.lower()
    fr_tensor = torch.tensor([fr_vocab[token] for token in fr_tokenizer(raw_fr_l)],
                            dtype=torch.long)
    en_tensor = torch.tensor([en_vocab[token] for token in en_tokenizer(raw_en_l)],
                            dtype=torch.long)
    if len(fr_tensor) <= MAX_LEN-2 and len(en_tensor) <= MAX_LEN-2:
        data.append((fr_tensor, en_tensor))
  return data

In [10]:
train_data = data_process(train_filepaths)
val_data = data_process(val_filepaths)
test_data = data_process(test_filepaths)

In [11]:
BATCH_SIZE = 128
PAD_IDX = fr_vocab['<pad>']
SOS_IDX = fr_vocab['<sos>']
EOS_IDX = fr_vocab['<eos>']

In [12]:
def generate_batch(data_batch):
  
    fr_batch, en_batch = [], []
    for (fr_item, en_item) in data_batch:
          en_batch.append(torch.cat([torch.tensor([SOS_IDX]), en_item, torch.tensor([EOS_IDX])], dim=0))
          fr_batch.append(torch.cat([torch.tensor([SOS_IDX]), fr_item, torch.tensor([EOS_IDX])], dim=0))
    en_batch = pad_sequence(en_batch, padding_value=PAD_IDX)
    fr_batch = pad_sequence(fr_batch, padding_value=PAD_IDX)
    fix=torch.ones(MAX_LEN,en_batch.shape[1])
    two= pad_sequence([fr_batch,en_batch, fix], padding_value=PAD_IDX)
    fr_batch=two[:,0,]
    en_batch=two[:,1,]
    return fr_batch, en_batch

In [13]:
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE,
                        shuffle=False, collate_fn=generate_batch)
valid_loader = DataLoader(val_data, batch_size=BATCH_SIZE,
                        shuffle=False, collate_fn=generate_batch)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=False, collate_fn=generate_batch)

In [14]:
# Get the input and the output sizes for model
input_size = len(fr_vocab)
output_size = len(en_vocab)
print (input_size,output_size)

6556 6192


## Train and Evaluate functions

In [ ]:
def train(model, dataloader, optimizer, criterion, scheduler=None, device='cpu'):
    model.train()

    # Record total loss
    total_loss = 0.

    # Get the progress bar for later modification
    progress_bar = tqdm_notebook(dataloader, ascii=True)

    # Mini-batch training
    for batch_idx, data in enumerate(progress_bar):
        source = data[0].transpose(1, 0).to(device)
        target = data[1].transpose(1, 0).to(device)

        translation = model(source)
        translation = translation.reshape(-1, translation.shape[-1])
        target = target.reshape(-1)

        optimizer.zero_grad()
        loss = criterion(translation, target)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        progress_bar.set_description_str(
            "Batch: %d, Loss: %.4f" % ((batch_idx + 1), loss.item()))

    return total_loss, total_loss / len(dataloader)

In [ ]:
def evaluate(model, dataloader, criterion, device='cpu'):
    # Set the model to eval mode to avoid weights update
    model.eval()
    total_loss = 0.
    with torch.no_grad():
        # Get the progress bar
        progress_bar = tqdm_notebook(dataloader, ascii=True)
        for batch_idx, data in enumerate(progress_bar):
            source = data[0].transpose(1, 0).to(device)
            target = data[1].transpose(1, 0).to(device)

            translation = model(source)
            translation = translation.reshape(-1, translation.shape[-1])
            target = target.reshape(-1)

            loss = criterion(translation, target)
            total_loss += loss.item()
            progress_bar.set_description_str(
                "Batch: %d, Loss: %.4f" % ((batch_idx + 1), loss.item()))

    avg_loss = total_loss / len(dataloader)
    return total_loss, avg_loss


## Train the LSTM seq2seq Model

In [ ]:
from seq2seq import Encoder, Decoder, Seq2Seq

In [ ]:
# Hyperparameters. You are welcome to modify these
encoder_emb_size = 128
encoder_hidden_size = 128
encoder_dropout = 0.2

decoder_emb_size = 128
decoder_hidden_size = 128
decoder_dropout = 0.2

learning_rate = 1e-3
model_type = "LSTM"

EPOCHS = 40

#input size and output size
input_size = len(fr_vocab)
output_size = len(en_vocab)

In [ ]:
encoder = Encoder(input_size, encoder_emb_size, encoder_hidden_size, decoder_hidden_size, dropout = encoder_dropout, model_type = model_type)
decoder = Decoder(decoder_emb_size, encoder_hidden_size, encoder_hidden_size, output_size, dropout = decoder_dropout, model_type = model_type)
seq2seq_model = Seq2Seq(encoder, decoder, device, attention=True)
optimizer = optim.Adam(seq2seq_model.parameters(), lr = learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

In [ ]:
for epoch_idx in range(EPOCHS):
    print("-----------------------------------")
    print("Epoch %d" % (epoch_idx+1))
    print("-----------------------------------")
    
    train_loss, avg_train_loss = train(seq2seq_model, train_loader, optimizer, criterion, device=device)
    scheduler.step(train_loss)

    val_loss, avg_val_loss = evaluate(seq2seq_model, valid_loader, criterion, device=device)
    
    print("Training Loss: %.4f. Validation Loss: %.4f. " % (avg_train_loss, avg_val_loss))
    print("Training Perplexity: %.4f. Validation Perplexity: %.4f. " % (np.exp(avg_train_loss), np.exp(avg_val_loss)))

-----------------------------------
Epoch 1
-----------------------------------


<ipython-input-20-0d1372a72a6c>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  progress_bar = tqdm_notebook(dataloader, ascii=True)


  0%|          | 0/176 [00:00<?, ?it/s]

<ipython-input-15-124b11f0a84d>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  progress_bar = tqdm_notebook(dataloader, ascii=True)


  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

Training Loss: 2.1451. Validation Loss: 2.6023. 
Training Perplexity: 8.5427. Validation Perplexity: 13.4946. 
-----------------------------------
Epoch 40
-----------------------------------


  0%|          | 0/176 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

Training Loss: 2.1294. Validation Loss: 2.5962. 
Training Perplexity: 8.4102. Validation Perplexity: 13.4133. 


## Train the transformer model

In [ ]:
from Transformer import TransformerTranslator

In [ ]:
learning_rate = 1e-4
EPOCHS = 40
hidden_dim=400
num_heads=10
dim_feedforward=2048
dim_k=96
dim_v=96
dim_q=96
max_length=50
#max_length = 43 

trans_model = TransformerTranslator(input_size, output_size, device,num_heads = num_heads, max_length = max_length, hidden_dim = hidden_dim ,dim_feedforward = dim_feedforward).to(device)

# optimizer = optim.Adam(model.parameters(), lr = learning_rate)
optimizer = torch.optim.Adam(trans_model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

In [ ]:
for epoch_idx in range(EPOCHS):
    print("-----------------------------------")
    print("Epoch %d" % (epoch_idx+1))
    print("-----------------------------------")
    
    train_loss, avg_train_loss = train(trans_model, train_loader, optimizer, criterion, device=device)
    scheduler.step(train_loss)

    val_loss, avg_val_loss = evaluate(trans_model, valid_loader, criterion, device=device)

    print("Training Loss: %.4f. Validation Loss: %.4f. " % (avg_train_loss, avg_val_loss))
    print("Training Perplexity: %.4f. Validation Perplexity: %.4f. " % (np.exp(avg_train_loss), np.exp(avg_val_loss)))

-----------------------------------
Epoch 1
-----------------------------------


<ipython-input-20-0d1372a72a6c>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  progress_bar = tqdm_notebook(dataloader, ascii=True)


  0%|          | 0/176 [00:00<?, ?it/s]

<ipython-input-15-124b11f0a84d>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  progress_bar = tqdm_notebook(dataloader, ascii=True)


  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

Training Loss: 1.8469. Validation Loss: 2.8045. 
Training Perplexity: 6.3401. Validation Perplexity: 16.5185. 
-----------------------------------
Epoch 40
-----------------------------------


  0%|          | 0/176 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

Training Loss: 1.8207. Validation Loss: 2.8050. 
Training Perplexity: 6.1764. Validation Perplexity: 16.5273. 


## Get some translations

In [ ]:
def translate(model, dataloader):
    model.eval()
    with torch.no_grad():
        # Get the progress bar 
        progress_bar = tqdm(dataloader, ascii = True)
        for batch_idx, data in enumerate(progress_bar):
            source = data[0].transpose(1,0).to(device)
            target = data[1].transpose(1,0).to(device)

            translation = model(source)
            return target, translation

In [ ]:
#model = seq2seq_model
model = trans_model

In [ ]:
target, translation = translate(model, test_loader)

  0%|          | 0/7 [00:00<?, ?it/s]


In [ ]:
raw = np.array([list(map(lambda x: en_vocab.get_itos()[x], target[i])) for i in range(target.shape[0])])

In [ ]:
raw[10:19]

array([['<sos>', 'three', 'people', 'sit', 'in', 'a', 'cave', '.', '\n',
        '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>',
        '<pad>', '<pad>', '<pad>', '<pad>'],
       ['<sos>', 'a', 'girl', 'in', 'a', 'jean', 'dress', 'is',
        'walking', 'along', 'a', 'raised', 'balance', 'beam', '.', '\n',
        '<eos>', '<pad>', '<pad>', '<pad>'],
       ['<sos>', 'a', 'blond', 'holding', 'hands', 'with', 'a', 'guy',
        'in', 'the', 'sand', '.', '\n', '<eos>', '<pad>', '<pad>',
        '<pad>', '<pad>', '<pad>', '<pad>'],
       ['<sos>', 'the', 'person', 'in', 'the', 'striped', 'shirt', 'is',
        'mountain', 'climbing', '.', '\n', '<eos>', '<pad>', '<pad>',
        '<pad>', '<pad>', '<pad>', '<pad>', '<pad>'],
       ['<sos>', 'two', 'men', 'pretend', 'to', 'be', '<unk>', 'while',
        'women', 'look', 'on', '.', '\n', '<eos>', '<pad>', '<pad>',
        '<pad>', '<pad>', '<pad>', '<pad>'],
       ['<sos>', 'people', 'standing', 'outside', 'of', 'a', 'b

In [ ]:
token_trans = np.argmax(translation.cpu().numpy(), axis = 2)
translated = np.array([list(map(lambda x: en_vocab.get_itos()[x], token_trans[i])) for i in range(token_trans.shape[0])])

In [ ]:
translated[10:19]

array([['<sos>', 'three', 'people', 'are', 'sitting', 'in', 'a', 'cave',
        '.', '\n', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>',
        '<eos>', '<eos>', '<eos>', '<eos>'],
       ['<sos>', 'a', 'girl', 'in', 'a', 'a', 'dress', 'is', 'is', 'a',
        'on', 'beam', 'expanse', 'a', '\n', '\n', '\n', '<eos>', '<eos>',
        '<eos>'],
       ['<sos>', 'a', 'blond', 'giving', 'the', 'hand', 'a', 'in', 'guy',
        '.', '\n', '\n', '\n', '<eos>', '<eos>', '<eos>', '<eos>',
        '<eos>', '<eos>', '<eos>'],
       ['<sos>', 'the', 'person', 'in', 'a', 'striped', 'is', '.', '.',
        '.', '.', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>',
        '<eos>', '<eos>', '<eos>'],
       ['<sos>', 'two', 'men', 'are', 'pretending', 'look', 'be', 'look',
        'connected', 'are', 'while', 'watch', '.', '.', '<eos>', '<eos>',
        '<eos>', '<eos>', '<eos>', '<eos>'],
       ['<sos>', 'people', 'standing', 'standing', 'in', 'of', 'of',
        'building', '\n', 